# Attributing vector features (automated)

**Original code:** [Alexandros Korkovelos](https://github.com/akorkovelos) <br />

This notebook employs a number of functions that extract values from raster layers and attribute them to the vector layer generated by the `"Creating clustered polygons from ascii grid.ipynb"`.

You will need the following layers:
* Any continuous of categorical raster layer in *.tif* format (placed inside the input directory)
* Clustered polygon vector layer in *.shp* or *.gpkg* -- (placed in the output directory)

**Note!** Layers can be added based on the mandates of the analysis..

## Importing necessary packages | defining functions

In [17]:
# Spatial
import geopandas as gpd
import rasterio
import rasterio.fill
from rasterstats import zonal_stats
from geojson import Feature, Point, FeatureCollection
import json

# numeric
import numpy as np

# System or other
import os
from IPython.display import display
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog, messagebox
import datetime

import warnings
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

In [18]:
country_name = "eth"     # suggent using UN 3 letter ISO code

In [19]:
# Directories
ROOT_DIR = os.path.abspath(os.curdir)
in_path = os.path.join(ROOT_DIR, country_name + "\\"+ 'input')
out_path = os.path.join(ROOT_DIR, country_name + "\\"+ 'output')

## name of layers
vect_nm = "{}_vector_admin1_clusters.gpkg".format(country_name)              # name of the layer containing the cluster/vector data
out_nm = "{}_vector_admin1_clusters_with_attributes".format(country_name)    # name of the output layer

In [20]:
# Processing Continuous/Numerical Rasters
def processing_raster_con(path, raster, prefix, method, clusters):
    """
    This function calculates stats for numerical rasters and attributes them to the given vector features. 
    
    INPUT: 
    name: string used as prefix when assigning features to the vectors
    method: statistical method to be used (check documentation)
    clusters: the vector layer containing the clusters
    
    OUTPUT:
    geojson file of the vector features including the new attributes
    """

    raster=rasterio.open(path + '\\' + raster)
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=prefix, geojson_out=True, all_touched=True)
    
    print("{} processing completed at".format(prefix), datetime.datetime.now())
    return clusters

In [21]:
## Processing Categorical/Discrete Rasters
def processing_raster_cat(path, raster, prefix, clusters):
    """
    This function calculates stats for categorical rasters and attributes them to the given vector features. 
    
    INPUT: 
    path: the directory where the raster layer is stored 
    raster: the name and extention of the raster layer 
    prefix: string used as prefix when assigning features to the vectors
    clusters: the vector layer containing the clusters
    
    OUTPUT:
    geojson file of the vector features including the new attributes
    """    
    raster=rasterio.open(path + '\\' + raster)
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        categorical=True,
        prefix=prefix, geojson_out=True, all_touched=True)
    
    print("{} processing completed at".format(prefix), datetime.datetime.now())
    return clusters

In [22]:
## Converting geojson to geodataframe
def geojson_to_gdf(workspace, geojson_file):
    """
    This function returns a geodataframe for a given geojson file
    
    INPUT: 
    workplace: working directory
    geojson_file: geojson layer to be convertes
    crs: projection system in epsg format (e.g. 'EPSG:32637')
    
    OUTPUT:
    geodataframe
    """
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(geojson_file)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    os.remove(output)
    
    print("cluster created anew at", datetime.datetime.now())
    return clusters

## Import vector features as geodataframes       

In [23]:
# Create a new geo-dataframe for vector features
clusters = gpd.read_file(out_path + "\\" + vect_nm)

## Collect raster names and type from directory

In [24]:
# Read files with tif extension and assign their name into two list for discrete and continuous datasets
raster_files_dis = []
raster_files_con =[]

for i in os.listdir(in_path):
    if i.endswith('.tif'):
        with rasterio.open(in_path + '\\' + i) as src:
            data = src.read() 
            unique_val = len(np.unique(data))
            if unique_val < 20:                                   # This value is arbitrary
                raster_files_dis.append(i)
            else:
                raster_files_con.append(i)

print ("We have identified {} continuous raster(s):".format(len(raster_files_con)),"\n",)
for raster in raster_files_con:
    print ( "*", raster)
    
print ("\n", "We have identified {} discrete raster(s):".format(len(raster_files_dis)),"\n",)
for raster in raster_files_dis:
    print ( "*", raster)

We have identified 7 continuous raster(s): 

* cwd_mai_irr_hig_bas.tif
* cwd_whe_rai_int_bas.tif
* evt_mai_irr_hig_bas.tif
* evt_whe_rai_int_base.tif
* rai_bas.tif
* yie_whe_irr_hig_bas.tif
* yie_whe_rai_int_bas.tif

 We have identified 1 discrete raster(s): 

* LCType.tif


## Extract raster values 

### Continuous datasets (e.g. rainfall, yield, evapotranspiration, GHI etc.)

In [25]:
for raster in raster_files_con:
    prefix = raster.strip(".tif")
    
    # Calling the extraction function for discrete layers
    clusters = processing_raster_con(in_path, raster, prefix, "mean", clusters)

cwd_mai_irr_hig_bas processing completed at 2020-12-13 17:41:36.451253
cwd_whe_rai_int_bas processing completed at 2020-12-13 17:41:40.016291
evt_mai_irr_hig_bas processing completed at 2020-12-13 17:41:43.578883
evt_whe_rai_int_base processing completed at 2020-12-13 17:41:47.175535
rai_bas processing completed at 2020-12-13 17:41:50.810224
yie_whe_irr_hig_bas processing completed at 2020-12-13 17:41:54.428060
yie_whe_rai_int_bas processing completed at 2020-12-13 17:41:58.012555


### Discrete datasets (e.g. Land cover type)

Note that the Land Cover layer used follows `International Geosphere-Biosphere Programme (IGBP) classification` (see [here](https://smap.jpl.nasa.gov/system/internal_resources/details/original/284_042_landcover.pdf) for more info).

In [26]:
for raster in raster_files_dis:
    prefix = raster.strip(".tif")
    
    # Calling the extraction function for discrete layers
    clusters = processing_raster_cat(in_path, raster, prefix, clusters)

LCType processing completed at 2020-12-13 17:42:13.116222


## Converting the geojson file to geodataframe

**NOTE** In case you get an Driver Error for reading the geojson file into a geodataframe, this might be cause due to attribution of "inf" or "-inf" value in one of the attributes. This is related to the way python handles json (see fix [here](https://stackoverflow.com/questions/17503981/is-there-a-way-to-override-pythons-json-handler)). An "easy" fix is that you import the geojson into Qgis and replace the erroneous value(s) manually. This is not ideal but it will do the job. In that case, save the updated geojson file and use the second (commented) line below to import into a geodataframe.

In [27]:
clusters = geojson_to_gdf(out_path, clusters)

cluster created anew at 2020-12-13 17:42:18.482765


## Exporting the geodataframe as vector layer

In [28]:
# Export as shapefile 
clusters.to_file(os.path.join(out_path,"{c}.gpkg".format(c=out_nm)),driver="GPKG")